<a href="https://colab.research.google.com/github/juanlu29/juanlu29/blob/gp_aprendizaje/libreriaGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Modulos y constantes

import numpy as np
import random
import scipy
import scipy.linalg
from matplotlib import pyplot as plt
import seaborn as sns
from google.colab import drive
import pandas as pd

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Pruebas con funciones anonimas en python

class funcionesPrueba():
  def __init__(self):
    '''
    Covarianza puede ser una cadena de texto o una funcion anónima como combinación de metodos de la clase
    covarianza es un diccionario
    '''
    self.covarianzas = self.kernels()


  def compCov(self,covarianza,**hiper):
    '''
    Permite definir el kernel a usar por el proceso gaussiano, tanto si es una de los kernels definidos en la clase como si es una combinacion de los mismos
    El argumento es una funcion anónima en caso de ser una composición, en caso contrario es suficiente con proporcionar el nombre del kernel definido
    hiper son los hiperparámetros que precisan el kernel y vienen dado por un diccionario
    '''
    self.Cov = lambda x_test, x_obs, **hiper: covarianza(x_test,x_obs,**hiper) # Covarianza definida

  class kernels():
    '''
    Se definen kernels de uso común junto a sus hiperparámetros
    '''
    def __init__(self):
      return

    def funcion_1(self,x_test,x_obs,**hiper):
      try:
        h_l = hiper.get("l")
        print(h_l)
      except:
        raise NameError("No se ha especificado el hiperparametro l")

      return (x_test*x_obs)/h_l

    def funcion_2(self,x_test,x_obs,**hiper):
      try:
        h_s = hiper.get("s")
        print(h_s)
      except:
        raise NameError("No se ha especificado el hiperparametro s")

      return h_s/(x_test*np.power(x_obs,2))


In [ ]:
x_test = np.linspace(1.,10.,5)
x_obs = np.linspace(1.,5.,5)
prueba = funcionesPrueba()
prueba.compCov(lambda x,y, **hiper: prueba.covarianzas.funcion_2*prueba.covarianzas.funcion_1,l=4.,s=3.)
print(prueba.Cov(x_test,x_obs))

TypeError: ignored

In [ ]:
class espacioPropio():
  '''
  Esta clase define el espacio propio donde proyectamos y la matriz de covarianza resultante
  '''

  def maternCov_nu_halfIntegers(self,p,x, x_train,s,l):
    '''
    Expresión simplificada de la covarianza de matérn para el caso que nu sean semienteros

    s es factor de escala del kernel. Permite escalar las trayectorias a la amplitud de la señal
    '''

    x_domain = np.concatenate((x_train,x), axis =None)

    r  = np.asarray([ [np.abs(xi-xj) for xi in x_domain] for xj in x_domain])

    ni = np.arange(p+1) # i corre desde 0 hasta el valor de p mismo

    # Factor escalar de factoriales
    gammaQuo = np.math.factorial(p)/np.math.factorial(2*p)

    # El siguiente ndarray tiene dimensionalidad 3, porque estamos generando un array a partir de una lista de matrices
    summatoryArray = np.asarray([(np.math.factorial(p + i)/(np.math.factorial(i)*np.math.factorial(p-i)))*np.power((np.sqrt(8*(p+0.5)*r)/l),p-i) for i in ni ])
    summatory = np.sum( np.asarray(summatoryArray), axis = 0) # Axis 0 significa que solo suma corriendo el indice de la posicion cero en el array, en este caso tendriamos un array tridimensional (x,y,z) y 0 hace referencia a x

    # Termino exponencial de los elementos del array
    exponential = np.exp(-(np.sqrt(2.*(p+0.5))*r)/l)

    K_total = s*gammaQuo*(np.multiply(exponential,summatory)) # Al ser un kernel estacionario, podemos interpretarlo como la evaluacion de una función dependiente de la distancia entre puntos del input

    plt.imshow(K_total)

    K0, K1, K2 = self.submatrices(K_total, len(x_train),len(x_train))

    return K2, K1.T, K0, x, x_train, np.zeros(len(x))

  def cov_drive(self,x,x_train, sigma_b,sigma_m):
    '''
    Obtiene la matriz de covarianza de un directorio en google drive
    '''
    cov_df = pd.read_csv('K_OU_experimental.csv')
    K_total_x = cov_df.to_numpy()

    K_total = K_total_x[:,1:]
    K_total = np.array(K_total, dtype=float)

    K0, K1, K2 = self.submatrices(K_total, len(x_train),len(x_train))

    return K2, K1.T, K0, x, x_train, np.zeros(len(x))

  def linear(self,x,x_train, sigma_b,sigma_m):
    '''
    Se interpretan sigma_b y sigma_m como varianzas de distribuciones a priori gaussianas
    para la ordenada en origen y para la pendiente de nuestro modelo linear.
    x es el input
    '''
    phi_x = np.array([np.array([1,xe]) for xe in x])
    phi_t = np.array([np.array([1,xe]) for xe in x_train])
    eps = np.array([[sigma_b,0],[0,sigma_m]])
    K_prior = np.matmul(np.matmul(phi_x,eps),phi_x.T)
    K_pred_train = np.matmul(np.matmul(phi_x,eps),phi_t.T) # Es la matriz de covarianzas entre datos de la distribución a priori y el entrenamiento
    K_train_train = np.matmul(np.matmul(phi_t,eps),phi_t.T) # Es la matriz de covarianzas entre los datos de entrenamiento

    return K_prior,K_pred_train,K_train_train, phi_x, phi_t, np.zeros(x.size)

  def covarianzaProceso(self,phi_x,covM):
    '''
    Devuelve la matriz de covarianza correspondiente a distribucion de probabilidad a priori en base a la incertidumbre de los parametros de una regresion lineal .
    '''
    return np.matmul(np.matmul(phi_x,covM),phi_x.T)

  def covarianzaAleatoria(self,x, x_train,sigma):
    '''
    Genera matriz de covarianza con correlaciones aleatorias sobre el espacio en que realizamos el proceso gaussiano,
    generamos una matriz de la aplicacion aleatoria dada una distribucion normal Sigma ~ normal(0,1)

    Partimos de que a priori, sin condicionar resultados, si tenemos los puntos sobre los que samplear las funciones
    y los de entrenamiento, partimos de que la matriz de covarianza a priori total o global resulta de combinar las siguientes submatrices

                                                  ( M(n_star,n_star), M(n_star,n_train) 
    M(n_star + n_train, n_star + n_train) =    (                                          )
                                                  ( M(n,train,n_star), M(n_train,n_train)

    Que es generada recordando la siguiente propiedad de matrices definidas positivas simétricas

    Cov = M*M_t

    Simplemente debemos generar M y obtener la matriz de covarianza de la anterior forma y las submatrices de covarianza cuando queramos condicionar el procesos gausiano

    Creo que no importa el orden de las coordenadas porque estamos trabajando con algebra lineal (permutaciones de filas dejan la matriz con las mismas propiedades)

    '''

    phi_x = x
    phi_t = x_train(97,)
    sqrt_K_total = np.array([[random.gauss(0,0.1) for xi in np.concatenate((x, x_train), axis=None)] for xj in np.concatenate((x, x_train), axis=None)])

    K_total = np.matmul(sqrt_K_total,sqrt_K_total.T) # Definiendola así nos aseguramos que resulte una matriz definida positiva
   
    K_prior = K_total[:x.size,:x.size]
    K_pred_train = K_total[:x.size,x.size:]
    K_train_train = K_total[x.size:,x.size:]

    return K_prior,K_pred_train,K_train_train, phi_x, phi_t, np.zeros(x.size)



  def corr_linear_creciente(self,x, x_train,s,l):
    '''
    Genera matriz de covarianza con correlaciones que aumentan con la separación entre coordenadas
    sigma y l son hiperparámetros de este kernel
    '''

  
    phi_x = x
    phi_t = x_train

    x_domain = np.concatenate((x,x_train), axis =None)

    K_total =  np.array([[(s*np.abs(xj-xi))/l for xi in x_domain] for xj in x_domain]) # Debemos tomar valor absoluto para asegurarnos que la matriz resultante sea simétrica

    #K_total = np.matmul(sqrt_K_total,sqrt_K_total.T) # Definiendola así nos aseguramos que resulte una matriz definida positiva    

    K0, K1, K2 = self.submatrices(K_total, x.size, x_train.size)

    return K0, K1, K2, phi_x, phi_t, np.zeros(x.size)

  def corr_linear_decreciente(self,x, x_train,s,l):
    '''
    Genera matriz de covarianza con correlaciones que disminuyen con la separación entre coordenadas
    sigma y l son hiperparámetros de este kernel

    cor(dx) =   s. / (1 + dx/l)

    '''

  
    phi_x = x
    phi_t = x_train

    x_domain = np.concatenate((x,x_train), axis =None)

    K_total =  np.array([[s/(1.+(np.abs(xj-xi)/l)) for xi in x_domain] for xj in x_domain]) # Debemos tomar valor absoluto para asegurarnos que la matriz resultante sea simétrica

    #K_total = np.matmul(sqrt_K_total,sqrt_K_total.T) # Definiendola así nos aseguramos que resulte una matriz definida positiva    

    K0, K1, K2 = self.submatrices(K_total, len(x), len(x_train))

    return K0, K1, K2, phi_x, phi_t, np.zeros(len(x))


  def kernel_exponencial_cuadratico(self,x, x_train,s,l):
    '''
    Genera matriz de covarianza con correlaciones de acuerdo al kernel exponencial cuadrático

    K(x-x') =   s² exp((-[x-x']²)/2l²)

    '''

    x_domain = np.concatenate((x_train,x), axis =None)

    K_total =  np.array([[np.power(s,2)*np.exp(-np.power(xi-xj,2)/(2.*np.power(l,2)))  for xi in x_domain] for xj in x_domain]) # Debemos tomar valor absoluto para asegurarnos que la matriz resultante sea simétrica


    # Se calculan derivadas parciales respecto los hiperparámetros del modelo

    #s
    K_total_part_d_s = np.array([[2.*s*np.exp(-np.power(xi-xj,2)/(2.*np.power(l,2)))  for xi in x_domain] for xj in x_domain]) 
    
    #l
    K_total_part_d_l = np.array([[-(np.power(xi-xj,2)/np.power(l,3))*np.power(s,2)*np.exp(-np.power(xi-xj,2)/(2.*np.power(l,2)))  for xi in x_domain] for xj in x_domain]) 


    K0, K1, K2 = self.submatrices(K_total, len(x_train),len(x_train))

    K_p_s_0, K_p_s_1, K_p_s_2 = self.submatrices(K_total_part_d_s, len(x_train),len(x_train))
    K_p_l_0, K_p_l_1, K_p_l_2 = self.submatrices(K_total_part_d_l, len(x_train),len(x_train))


    return K2, K1.T, K0,  K_p_s_0, K_p_s_1, K_p_s_2, K_p_l_0, K_p_l_1, K_p_l_2, x, x_train, np.zeros(len(x))

  def submatrices(self,K_total,i,f):
    '''
    Este metodo, dada una matriz de covarianzas del proceso gausiano sin condicionar a las observaciones,
    devuelve las matrices K_prior, K_pred_train, K_train_train que serán usadas para muestrear el proceso

    Solo sirve si el dominio de prueba y el de entrenamiento estan concatenados secuencialmente (no mezclados u ordenados de otra manera los puntos)
    '''
    Ks = [K_total[:i,:i], K_total[:i,i:], K_total[i:,i:]]
    return Ks[0], Ks[1], Ks[2] #  K_train_train, K_train_pred, K_pred_pred o K_prior prior respectivamente



class gaussProcess():
  def __init__(self,K_p,K_p_t,K_t_t,prior_Partial_Der,Mean, Input, noiseLevel):
    '''
    Los objetos de esta clase modelan procesos gaussianos caracterizados por su promedio y covarianza
    Eps es la matriz de covarianzas de los pesos del proceso gausiano
    prior_Partial_der corresponde a una lista de matrices de la distribución a priori correspondientes a su derivada respecto los distintos parámetros
    '''
    self.x = Input # Dominio de la regresion
    self.K_prior = K_p # Matriz de covarianzas usada para definir la distribucion a priori y mas tarde para muestrear funciones del proceso gausiano
    self.K_pred_train = K_p_t # Es la matriz de covarianzas entre datos de la distribución a priori y el entrenamiento
    self.K_train_train = K_t_t # Es la matriz de covarianzas entre los datos de entrenamiento
    self.mean_prior = Mean
    self.Derivadas = prior_Partial_Der
    self.L_prior = self.cholDescomp(self.K_prior)
    self.Ruido = noiseLevel  # amplitud del ruido

    class covarianzas():
      def __init__(self,modelos):
        self.covarianzas_usadas = modelos # Lista 

  def cholDescomp(self,K):
    '''
    Cholesky decomposition
    '''
    try:
      L = scipy.linalg.cholesky(K, lower=True)
    except:
      L = scipy.linalg.cholesky(K + np.diag(0.000001*np.ones(int(np.sqrt(K.size)))), lower=True)

    return L

  def calcInvK(self,matriz):
    '''
    Calculo de la inversa de matriz. Pensado para ser la inversa de la matriz de covarianza.
    '''
    # Calculo de matriz inversa de la covarianza. 
    try:
      self.invK = np.linalg.inv(matriz)
    except:
      print("La matriz inversa no existe porque el determinante es cero")



  def condicionarGP(self,entrenamiento_x,entrenamiento_y):
    '''
    Dadas unas observaciones junto a sus valores en el dominio de los procesos, obtiene la distribución condicionada a las trayectorias asociadas a dichas medidas.
    También calcula el logaritmo de la probabilidad de verosimilitud marginal (marginal likelihood) de las observaciones dados los inputs 
    y parámetros del modelo usado.

    Este algoritmo esta especificado en el libro "Gaussian Processes for Machine Learning", como algoritmo 2.1
    '''
    self.xtrain = entrenamiento_x
    self.ytrain = entrenamiento_y
    self.n_train = len(entrenamiento_x)

    # Definimos nueva covarianza
    # Primero se realiza la descomposición cholesky de la adición de la covarianza de la distribución a priorir y el término de ruido.
    cholL = np.linalg.cholesky(self.K_train_train + np.power(self.Ruido,2)*np.diag(np.ones(len(self.xtrain))))

    # Calculo del vector alfa como solucion del sistema K*alfa = y_entrenamiento usando la descomposición cholesky anterior
    alfa = scipy.linalg.cho_solve((cholL,True),self.ytrain)

    # La media predictiva dados los datos
    self.mean_pred = np.matmul(self.K_pred_train,alfa)

    # Vector v solución particular del sistema cholL*v = k_star para todos los datos del entrenamiento.
    # Este sistema se resuelve introduciendo k_star no como matriz columna sino como la submatriz calculada como la evaluación de la covarianza a priori de los inputs de los valores a predecir y los inputs del entrenamiento
    v = scipy.linalg.solve_triangular(cholL,self.K_pred_train.T, lower=True)


    # Covarianza predictiva. El proceso llevado a cabo ha sido obtener la distribución condicionada del GP (inputs predictivos) a las observaciones y_entrenamiento
    self.K_pred = self.K_prior - np.matmul(v.T,v)

    # Magnitud importante para muestrear procesos gausianos.
    self.L_pred = self.cholDescomp(self.K_pred)


  def log_prob_verosimilitud_datos(self,entrenamiento_x,entrenamiento_y):
    '''
    Dadas unas observaciones, kernel e hiperparámetros dados, calcula el logaritmo de la probabilidad de verosimilitud de las observaciones al modelo y sus parámetros
    '''

    # Primero se realiza la descomposición cholesky de la adición de la covarianza de la distribución a priorir y el término de ruido.
    cholL = np.linalg.cholesky(self.K_train_train + np.power(self.Ruido,2)*np.diag(np.ones(len(entrenamiento_x))))

    # Calculo del vector alfa como solucion del sistema K*alfa = y_entrenamiento usando la descomposición cholesky anterior
    alfa = scipy.linalg.cho_solve((cholL,True),entrenamiento_y)

    # Calculo de la probabilidad de verosimilitud marginal a este modelo y sus parámetros
    self.log_marg_y = -np.sum(np.log(np.diag(cholL)),axis=0)
    self.log_marg_y = self.log_marg_y - 0.5*np.dot(alfa,entrenamiento_y)
    self.log_marg_y = self.log_marg_y - (float(len(entrenamiento_x)/2.))*np.log(2*pi)


  def distribucionGP(self,mean,L):
    '''
    Genera realizaciones del proceso gaussiano dada la descomposicion cholesky de la matriz de covarianzas y la media correspondiente
    '''
    gaussNumbers = np.fromiter([ random.gauss(0,1) for x in range(len(np.diag(L))) ],float)
    return mean + np.matmul(L,gaussNumbers.T)

  def sigmaCalc(self,cov):
    '''
    Desviacion estándar punto a punto del proceso gausiano dado por la matriz de covarianza 
    Es la raiz cuadrado de los elementos de la diagonal de la matriz de covarianzas del proceso generado
    '''
    return np.sqrt(np.diag(cov))


  def correlacionPuntoPunto(self):
    '''
    Obtiene la correlacion punto a punto dado un inputo concreto por el indice i_x
    '''
    try:
      self.corrPP_pred = np.array([ self.K_pred[i,:] for i in range(len(self.K_pred)) ])
    except:
      print(" Proceso gaussiano no entrenado aún, no existe K_pred ")
    self.corrPP_prior = np.array([ self.K_prior[i,:] for i in range(len(self.K_prior)) ])

  def derivadaVerosimilitudMarginal(self,*args):
    '''
    Dadas unas observaciones particulares estima la derivada del logaritmo de la probabilidad asociada a la distribución de verosimilitud marginal asociada al modelo respecto sus hiperparámetros
    '''

    if not isinstance(args[0], np.ndarray):
      raise NameError("Las observaciones dadas para calcular la derivada del logaritmo de la distribución marginal de verosimilitud no es un array con datos")
    else:
      observaciones = args[0]
      observaciones = observaciones.reshape(len(observaciones),1) # Esta operacion es necesaria para que mas tarde el producto matmul nos reproduzca una matriz en lugar de producto escalar entre vectores.

    # Vector auxiliar alpha
    try:
      alpha = np.matmul(self.invK,observaciones)
    except:
      raise NameError("Necesitas calcular la matriz inversa de covarianzas de la distribución a priori")

    alpha_alpha = np.matmul(alpha,alpha.T) 
    alpha_alpha_minus_invK = alpha_alpha - self.invK

    # Calculo. Es el valor de la derivada del logaritmo
    var_incSignal = self.Ruido*np.sum(alpha_alpha_minus_invK)# Si asumimos como hiperparámetro la amplitud del ruido sobre las medidas respecto la señal verdadera, se debe añadir este término
    self.derVerMarg = [ 0.5*np.trace(np.matmul(alpha_alpha_minus_invK,self.Derivadas[i])) for i in range(len(self.Derivadas))]
    self.derVerMarg.append(var_incSignal)
    self.derVerMarg = np.asarray(self.derVerMarg)
  
  def derivada_LO_CV(self,*args):
    '''
    Dadas unas observaciones particulares estima la derivada de la suma de los logaritmos de la evaluación de la distribución de probabilidad marginal de verosimilitud a que subconjuntos
    de las observaciones están condicionadas al resto, respecto a los hiperparámetros del modelo. Si se encuentra el máximo de este valor, estamos asegurándonos de que con los hiperparámetros ajustados
    las "predicciones" tras generar trayectorias del proceso gausiano que corresponderían a los subconjuntos excluidos sucesivamente son las más probables que el modelo puede hacer. Al ser la suma, esta optimización 
    es global y abarca la aproximación de observacion/test de todos los subconjuntos.
    '''

    if not isinstance(args[0], np.ndarray):
      raise NameError("Las observaciones dadas para calcular la derivada del logaritmo de la distribución marginal de verosimilitud no es un array con datos")
    else:
      observaciones = args[0]

    # Vector auxiliar alpha y matriz auxiliar Z y la diagonal de la inversa de la distribucion a priori
    try:
      alpha = np.matmul(self.invK,observaciones)
      invK_diag = np.diag(self.invK)
      zeta = [np.matmul(self.invK,self.Derivadas[i]) for i in range(len(self.Derivadas)) ] # Es una lista de matrices
      zeta_K = [np.diag(np.matmul(zeta[i],self.invK)) for i in range(len(self.Derivadas)) ]
      alpha_invK = np.multiply(alpha,1./invK_diag)
    except:
      raise NameError("Necesitas calcular la matriz inversa de covarianzas de la distribución a priori")
    
    termino_1 = [np.matmul(np.matmul(alpha_invK,zeta[i]),alpha) for i in range(len(self.Derivadas)) ] # Debido a Z esto es una lista
    termino_2 = [np.sum(np.multiply(np.multiply(0.5*(1. + np.multiply(np.power(alpha,2),1./invK_diag)),zeta_K[i]),1./invK_diag)) for i in range(len(self.Derivadas)) ] # Debido a zeta_K esto es una lista

    # Calculo. Es el valor de la derivada del logaritmo
    self.derVerLOO = np.asarray([ termino_1[i] + termino_2[i] for i in range(len(self.Derivadas))])
    

In [ ]:


class datosVisualizacion():
  '''
  Esta clase contiene los datos de entrenamiento y realizaciones de los procesos gaussianos para la visualización de los mismos
  '''
  def __init__(self,Titulo):
    self.x = []
    self.y = []
    self.sigma = []
    self.realizacion_dict = {}
    self.titulo = Titulo
    self.inicializacionPlot()

  def inicializacionPlot(self):
    '''
    Inicializa el objeto matplotlib
    '''
    self.fig,self.ax = plt.subplots(nrows=1,ncols=1,figsize=(10,10))
    self.ax.set_title(self.titulo)
    self.fig_hm, self.ax_hm = plt.subplots(nrows=1,ncols=1,figsize=(10,10))

  def add_covariance(self, cov):
    '''
    Añada matriz de covarianza y alternativamente calcula la covarianza del proceso gaussiano sobre el dominio
    '''

    self.cov_GP = cov

  def add_mean(self,X,Mean):
    '''
    Almacena la desviación estándar de un proceso gausiano dado
    '''
    self.x_mean = X
    self.mean = Mean


  def add_sigma(self,X,Sigma):
    '''
    Almacena la desviación estándar de un proceso gausiano dado
    '''
    self.x_sigma = X
    self.sigma = Sigma
    self.meanMarker = 'r-'

  def add_train_data(self,X,Y):
    self.xtrain = X
    self.ytrain = Y
    self.marker = 'b*'

  def add_data(self,X,Y,label):
    self.x.append(X)
    self.y.append(Y)
    self.realizacion_dict[label] = len(self.x)-1

  def add_corrPuntoPunto(self,corrPP, corrInputs):
    '''
    Añade correlaciones punto a punto sobre un dominio dado por los valores corrInputs
    '''
    self.corrPP = corrPP
    self.corr_in = corrInputs

  def add_data_plot(self,label):
    try:
      self.ax.plot(self.x[self.realizacion_dict[label]],self.y[self.realizacion_dict[label]],'g--')
    except:
      print('No existe datos referentes a etiqueta :',label)

  def add_train_data_plot(self):
    try:
      self.ax.plot(self.xtrain,self.ytrain,self.marker)
    except:
      print('No existen datos de entrenamiento')

  def add_mean_plot(self):
    try:
      self.ax.plot(self.x_mean,self.mean,self.meanMarker)
    except:
      print('No existen datos de entrenamiento')



  def contorno_sigma(self):
    '''
    Muestra en pantalla contorno de 1 y 2 veces los valores de sigma del proceso gausiano
    '''
    self.ax.fill(np.concatenate((self.x_sigma,self.x_sigma[::-1])),
             np.concatenate((self.mean - 1.9600 * self.sigma,
                             (self.mean + 1.9600 * self.sigma)[::-1])),
             alpha=.45, fc='y', ec='None', label='95% confidence interval')
    self.ax.fill(np.concatenate((self.x_sigma,self.x_sigma[::-1])),
             np.concatenate((self.mean - 1.000 * self.sigma,
                             (self.mean + 1.000 * self.sigma)[::-1])),
             alpha=.35, fc='b', ec='None', label='68% confidence interval')
    
  def plotCorrPuntoPunto(self,indices):
    '''
    Visualiza correlaciones punto a punto para los inputs dados por sus indices en la lista indices como argumento del metodo
    '''

    ' Generamos y visualizamos las imagen directamente en este metodo'
    self.fig_cpp, self.ax_cpp = plt.subplots(nrows=1,ncols=1,figsize=(10,10))
    try:
      [self.ax_cpp.plot(self.corr_in,self.corrPP[indice],'-') for indice in indices] 
    except:
      print('No existe datos referentes referentes a correlaciones punto a punto')
    self.ax_cpp.grid(True)
    self.fig_cpp.show()



  def mostrarPlot(self):
    '''
    Muestra plot
    '''
    self.add_mean_plot()
    self.add_train_data_plot()
    self.contorno_sigma()
    self.ax.grid(True)
    self.fig.show()

  def hmapCov(self):
    '''
    Visualiza covarianza como campo bidimensional
    '''
    plotRange = np.arange(len(self.cov_GP))
    heatmap = self.ax_hm.pcolormesh(self.cov_GP,cmap='RdBu_r')
    self.ax_hm.axis([plotRange.min(), plotRange.max(), plotRange.max(), plotRange.min()])
    self.fig_hm.colorbar(heatmap, ax=self.ax_hm)
    self.fig_hm.show()


